In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
from sklearn.linear_model import LogisticRegression

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [37]:
from sklearn.preprocessing import LabelEncoder

In [194]:
from string import punctuation

In [264]:
from sklearn.model_selection import train_test_split

In [4]:
## Abrir o arquivo para leitura, carregando um dataframe do arquivo

In [306]:
df = pd.read_csv( "../../../dados/nlp/news_sentiment_analysis.csv", encoding="utf-8")

In [308]:
df['Description'] 

0       ST. GEORGE — Kaitlyn Larson, a first-year teac...
1       Harare, Zimbabwe – Local businesses are grappl...
2       (marketscreener.com) Billionaire Elon Musk has...
3       (marketscreener.com) A U.S. trade regulator on...
4       4.5 million households in the U.S. have solar ...
                              ...                        
3495    QRG Capital Management Inc. increased its stak...
3496    QRG Capital Management Inc. bought a new posit...
3497    QRG Capital Management Inc. boosted its stake ...
3498    WESTFORD, Mass., July 18, 2024 /PRNewswire/ --...
3499    QRG Capital Management Inc. reduced its holdin...
Name: Description, Length: 3500, dtype: object

In [310]:
## Pre processamento do texto de description

In [312]:
texto = df["Description"][0]
texto

'ST. GEORGE — Kaitlyn Larson, a first-year teacher at Pine View High School, recently received the Best in State award for a business leadership course. Larson’s students in grades 10-12 had the highest passing rate on the YouScience exam during the spring and fall semesters of 2023. The YouScience exam is similar to final exams [&#8230;]'

In [362]:
pontuacoes = '!"#$%&()*+,/:;<=>?@[\\]^_{|}~'

In [364]:
def limpar_texto( texto : str ) -> str:
    mascara = str.maketrans( "", "", pontuacoes )
    texto_limpo = texto.translate( mascara ).lower()
    return texto_limpo

In [366]:
noticias = df[["Title", "Description", "Sentiment", "Type"]]

In [368]:
noticias

,Title,Description,Sentiment,Type
0,Pine View High teacher wins Best in State awar...,"ST. GEORGE — Kaitlyn Larson, a first-year teac...",positive,Business
1,Businesses Face Financial Strain Amid Liquidit...,"Harare, Zimbabwe – Local businesses are grappl...",neutral,Business
2,Musk donates to super pac working to elect Tru...,(marketscreener.com) Billionaire Elon Musk has...,positive,Business
3,US FTC issues warning to franchisors over unfa...,(marketscreener.com) A U.S. trade regulator on...,negative,Business
4,Rooftop solar's dark side,4.5 million households in the U.S. have solar ...,positive,Business
...,...,...,...,...
3495,"Arrow Electronics, Inc. (NYSE:ARW) Shares Purc...",QRG Capital Management Inc. increased its stak...,positive,Technology
3496,"3,120 Shares in NICE Ltd. (NASDAQ:NICE) Bought...",QRG Capital Management Inc. bought a new posit...,positive,Technology
3497,"QRG Capital Management Inc. Has $857,000 Stock...",QRG Capital Management Inc. boosted its stake ...,positive,Technology
3498,Biotechnology Market: Surging Investments and ...,"WESTFORD, Mass., July 18, 2024 /PRNewswire/ --...",neutral,Technology


In [370]:
noticias["Description"]

0       ST. GEORGE — Kaitlyn Larson, a first-year teac...
1       Harare, Zimbabwe – Local businesses are grappl...
2       (marketscreener.com) Billionaire Elon Musk has...
3       (marketscreener.com) A U.S. trade regulator on...
4       4.5 million households in the U.S. have solar ...
                              ...                        
3495    QRG Capital Management Inc. increased its stak...
3496    QRG Capital Management Inc. bought a new posit...
3497    QRG Capital Management Inc. boosted its stake ...
3498    WESTFORD, Mass., July 18, 2024 /PRNewswire/ --...
3499    QRG Capital Management Inc. reduced its holdin...
Name: Description, Length: 3500, dtype: object

In [372]:
noticias["descricao_limpa"] = noticias["Description"].apply(limpar_texto)
noticias["descricao_limpa"]

C:\Users\anton\AppData\Local\Temp\ipykernel_40852\1386010132.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  noticias["descricao_limpa"] = noticias["Description"].apply(limpar_texto)


0       st. george — kaitlyn larson a first-year teach...
1       harare zimbabwe – local businesses are grappli...
2       marketscreener.com billionaire elon musk has d...
3       marketscreener.com a u.s. trade regulator on f...
4       4.5 million households in the u.s. have solar ...
                              ...                        
3495    qrg capital management inc. increased its stak...
3496    qrg capital management inc. bought a new posit...
3497    qrg capital management inc. boosted its stake ...
3498    westford mass. july 18 2024 prnewswire -- acco...
3499    qrg capital management inc. reduced its holdin...
Name: descricao_limpa, Length: 3500, dtype: object

In [374]:
noticias

,Title,Description,Sentiment,Type,descricao_limpa
0,Pine View High teacher wins Best in State awar...,"ST. GEORGE — Kaitlyn Larson, a first-year teac...",positive,Business,st. george — kaitlyn larson a first-year teach...
1,Businesses Face Financial Strain Amid Liquidit...,"Harare, Zimbabwe – Local businesses are grappl...",neutral,Business,harare zimbabwe – local businesses are grappli...
2,Musk donates to super pac working to elect Tru...,(marketscreener.com) Billionaire Elon Musk has...,positive,Business,marketscreener.com billionaire elon musk has d...
3,US FTC issues warning to franchisors over unfa...,(marketscreener.com) A U.S. trade regulator on...,negative,Business,marketscreener.com a u.s. trade regulator on f...
4,Rooftop solar's dark side,4.5 million households in the U.S. have solar ...,positive,Business,4.5 million households in the u.s. have solar ...
...,...,...,...,...,...
3495,"Arrow Electronics, Inc. (NYSE:ARW) Shares Purc...",QRG Capital Management Inc. increased its stak...,positive,Technology,qrg capital management inc. increased its stak...
3496,"3,120 Shares in NICE Ltd. (NASDAQ:NICE) Bought...",QRG Capital Management Inc. bought a new posit...,positive,Technology,qrg capital management inc. bought a new posit...
3497,"QRG Capital Management Inc. Has $857,000 Stock...",QRG Capital Management Inc. boosted its stake ...,positive,Technology,qrg capital management inc. boosted its stake ...
3498,Biotechnology Market: Surging Investments and ...,"WESTFORD, Mass., July 18, 2024 /PRNewswire/ --...",neutral,Technology,westford mass. july 18 2024 prnewswire -- acco...


In [377]:
## Vetorizar os textos X e Y em vetores de numeros e numero categorico

In [379]:
vetorizador = TfidfVectorizer(max_features = 1000) 

In [381]:
bow = vetorizador.fit_transform( noticias['descricao_limpa'] ) 

In [383]:
vocabulario = vetorizador.get_feature_names_out()

In [385]:
bow_matrix = pd.DataFrame.sparse.from_spmatrix(bow, columns=vocabulario)

In [387]:
bow_matrix.shape

(3500, 1000)

In [389]:
bow_matrix

,00,10,100,11,12,13,13f,14,141st,15,...,year,yearly,years,yet,york,you,your,yourself,zu,über
0,0,0.20329,0,0,0.193791,0,0,0,0,0,...,0.167017,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3495,0,0,0,0,0,0,0.177388,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3496,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3497,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3498,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [391]:
encoder = LabelEncoder()

In [393]:
Y = encoder.fit_transform(noticias['Type'])
Y

array([0, 0, 0, ..., 6, 6, 6])

In [395]:
# treino_X = bow_matrix[0:2500]
# treino_Y = Y[0:2500]

In [397]:
# teste_X = bow_matrix[2500:]
# teste_Y = Y[2500:]

In [399]:
treino_X, teste_X, treino_Y, teste_Y = train_test_split(bow_matrix, Y, test_size = 0.3, random_state = 45)

In [401]:
model = LogisticRegression()

In [403]:
model.fit(treino_X, treino_Y)

LogisticRegression()

In [405]:
acuracia = model.score(teste_X, teste_Y)
acuracia

0.8704761904761905

### Acuracias

#### Com 19920 palavras => 0.843
#### Com 2000 palavras => 0.872
#### Com 1000 palavras => 0.881
#### Com 1000 palavras limpas => 0.869
#### Com 1000 palavras limpas usando train_test_split => 0.8609523809523809
#### Com 1000 palavras limpas (mantendo . - ' e `) usando train_test_split => 0.8704761904761905

In [103]:
texto = df["Description"][2197]
texto
texto = "Marine Le Pen seems politically dead after her guilty verdict. Can the far-right win in France without her?"

In [105]:
tipo = df["Type"][2197]
tipo

'Business'

In [107]:
vetorizador2 = TfidfVectorizer(vocabulary = vocabulario) 
bow2 = vetorizador2.fit_transform( [texto] ) 

In [109]:
bow_matrix2 = pd.DataFrame.sparse.from_spmatrix(bow2, columns=vocabulario)

In [111]:
bow_matrix2

,00,000,0000,001,003,005,009,01,010,011,...,überwiegen,überwiegend,überwogen,üblichen,üblicherweise,ünalm,čime,œuvres,španije,španski
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [115]:
saida = model.predict(bow_matrix2)
saida

array([5])

In [117]:
encoder.inverse_transform(saida)

array(['Sports'], dtype=object)